In [2]:
import pandas as pd

def serialize_csv_for_llm(file_path, max_rows=5):
    """
    Reads a CSV file and returns a clean string format for LLM prompts.
    Limits to `max_rows` to keep the input small.
    """
    df = pd.read_csv(file_path)
    sample = df.head(max_rows)
    
    # Create column header row
    columns_line = " | ".join(sample.columns)
    
    # Create data rows
    rows = []
    for _, row in sample.iterrows():
        row_str = " | ".join(str(cell) for cell in row)
        rows.append(row_str)

    # Combine everything
    table_text = f"Columns: {columns_line}\n\n"
    for idx, row in enumerate(rows):
        table_text += f"Row {idx+1}: {row}\n"
    
    return table_text


In [7]:
def generate_prompt(table_text, instruction):
    return f"""
You are a smart table transformer.

Task: {instruction}

Here is the input data (in table format):
{table_text}

Please provide the transformed data in the same format.
do not miss any data from the input file
"""


In [8]:
import requests

def call_mistral(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",
            "prompt": prompt,
            "temperature": 0.3,
            "stream": False
        }
    )
    return response.json()["response"].strip()


In [9]:
# Step 4: Full Pipeline Execution

if __name__ == "__main__":
    # ✅ Set your CSV file path here
    csv_path = r"C:\Users\sanka\OneDrive\Desktop\Tabulax\DATASETS\Tabulax datasets\ALL CSV\Customer-Churn-Records.csv"
    output_file_path = "transformed_output.txt"

    # Step 1: Serialize table
    table_text = serialize_csv_for_llm(csv_path)

    # Step 2: Define transformation instruction
    instruction = "Change all numerical values to the Indian Number System format (e.g., 1234567 → 12,34,567)."

    # Step 3: Generate prompt
    prompt = generate_prompt(table_text, instruction)

    # Step 4: Call Mistral and get response
    response = call_mistral(prompt)

    # Step 5: Save result to a file
    with open(output_file_path, "w", encoding="utf-8") as f:
        f.write(response)

    print(f"\n✅ Transformed output saved to: {output_file_path}")


✅ Transformed output saved to: transformed_output.txt
